<h1>2장 토큰과 임베딩</h1>
<i>LLM 구축의 핵심적인 부분인 토큰과 임베딩을 살펴 봅니다.</i>


<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter02.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 2장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>


---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

## 2장에서 다루는 내용

| 학습 요소       | 내용                          |
| ----------- | ------------------------------ |
| 토큰(Token)   | 텍스트가 모델 내부에서 다뤄지는 가장 작은 단위     |
| Subword 분절  | BERT, GPT 등 모델마다 다른 토큰화 방식을 비교 |
| 문맥 임베딩      | LLM이 각 단어를 문맥에 따라 벡터로 표현하는 원리  |
| 문장 임베딩      | SBERT로 문장 전체 의미를 벡터로 변환        |
| 단어 임베딩      | Word2Vec, GloVe 등 전통 NLP 모델 이해 |
| 벡터 유사도      | 단어/문장/노래 유사도 계산 가능             |
| Word2Vec 추천 | 간단한 추천 시스템 원리 이해               |


In [1]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [2]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

# LLM 다운로드하고 실행하기

빠른 추론을 위해 GPU에 모델을 로드합니다. 모델과 토크나이저를 개별적으로 살펴 보기 위해 따로따로 로드합니다.

| 항목             | 내용                                                      |
| -------------- | ------------------------------------------------------- |
| 모델 로딩          | `AutoModelForCausalLM.from_pretrained()`로 Phi-3 모델을 불러옴 |
| GPU 사용         | `device_map="cuda"` 를 통해 모델을 GPU에서 실행                   |
| dtype 자동 선택    | `torch_dtype="auto"` 로 성능과 메모리 효율을 자동 최적화               |
| remote code 허용 | `trust_remote_code=True`로 모델이 제공하는 커스텀 로직 사용 가능         |
| 토크나이저 로딩       | `AutoTokenizer.from_pretrained()` 로 동일 모델용 토크나이저 불러오기   |
| 전체 목적          | 사전학습된 LLM(Phi-3)을 로드해서 텍스트 생성 작업에 활용하기 위한 기본 구성         |


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# ---------------------------------------------------------------
# 1. 사용할 모델과 토크나이저 클래스를 불러옵니다. (약 2분 소요)
#    - AutoModelForCausalLM :
#        → '문장을 이어서 생성하는' 언어 모델을 로드할 때 사용
#    - AutoTokenizer :
#        → 텍스트를 숫자로 바꾸고, 숫자를 다시 텍스트로 바꾸는 도구
# ---------------------------------------------------------------

# ---------------------------------------------------------------
# 2. 사전학습(Pretrained) 언어 모델 불러오기
#    - 여기서는 Microsoft가 공개한 'Phi-3-mini-4k-instruct' 모델을 사용합니다.
#
#    매개변수 설명:
#    - device_map="cuda"
#        → GPU를 사용해 모델을 실행하겠다는 뜻
#
#    - torch_dtype="auto"
#        → 모델이 자동으로 가장 적합한 숫자 정밀도를 선택함
#        (GPU 성능 향상 및 메모리 절약 효과)
#
#    - trust_remote_code=True
#        → 모델이 제공하는 추가 코드를 신뢰하고 그대로 실행
#          (일부 모델은 자체적인 커스텀 로더가 필요하기 때문)
# ---------------------------------------------------------------
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# ---------------------------------------------------------------
# 3. 토크나이저 불러오기
#    - 같은 모델 이름을 사용해야 모델과 토크나이저가 서로 맞습니다.
#    - 토크나이저는 '문장 → 토큰(숫자)' 변환과
#                  '토큰 → 문장' 복원을 담당합니다.
# ---------------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added mal

| 항목                       | 설명                       |
| ------------------------ | ------------------------ |
| **프롬프트(prompt)**         | 모델에게 “무엇을 생성하라”는 요청 문장   |
| **tokenizer(prompt)**    | 문장을 숫자(token)로 바꾸는 과정    |
| **return_tensors="pt"**  | 결과를 PyTorch 텐서 형태로 반환    |
| **input_ids.to("cuda")** | 생성 속도를 높이기 위해 GPU에서 처리   |
| **model.generate()**     | 모델이 이어지는 새 문장을 생성        |
| **max_new_tokens=20**    | 생성되는 문장 길이 제한            |
| **tokenizer.decode()**   | 숫자 → 사람이 읽을 수 있는 텍스트로 변환 |


In [17]:
# 모델에게 보내줄 문장(프롬프트)을 작성합니다.
# 모델이 이 문장을 보고 이어지는 문장을 생성합니다.
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# --------------------------------------------------------
# 1. 프롬프트를 토큰으로 변환 (문장 -> 숫자)
#    - tokenizer()는 문장을 모델이 이해할 수 있는 숫자 배열로 바꿉니다.
#    - return_tensors="pt" : PyTorch 텐서 형태로 반환
#    - to("cuda") : GPU로 이동하여 빠르게 처리
# --------------------------------------------------------
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# --------------------------------------------------------
# 2. 모델이 텍스트를 생성하도록 요청
#    - generate() 는 모델에게 “문장을 이어서 써라”라고 지시하는 함수
#    - max_new_tokens=30 : 새로 생성할 최대 단어 수(토큰 수)
# --------------------------------------------------------
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=30
)

# --------------------------------------------------------
# 3. 숫자로 된 결과를 다시 자연어 문장으로 변환
# --------------------------------------------------------
print(tokenizer.decode(generation_output[0]))

Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap


Dear Sarah,


I hope this message finds


In [20]:
# 프롬프트(문장)가 토큰(token)이라는 숫자들로 변환된 결과를 출력
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001]], device='cuda:0')


In [21]:
# input_ids[0] : 프롬프트(문장)가 토큰 ID 배열로 변환된 것 중 첫 번째 문장
# 이 배열 안에는 문장을 구성하는 '숫자 토큰'들이 순서대로 들어 있습니다.

for id in input_ids[0]:
    # tokenizer.decode(id)
    #   → 숫자 하나(id)가 어떤 글자/단어 조각(subword)을 의미하는지 다시 텍스트로 변환
    #   → 즉, 토큰화된 결과를 사람이 읽을 수 있는 형태로 확인하는 과정
    print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.
<|assistant|>


In [22]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001,  3323,   622, 29901,   317,  3742,   406,
          6225, 11763,   363,   278, 19906,   292,   341,   728,   481,    13,
            13,    13, 29928,   799, 19235, 29892,    13,    13,    13, 29902,
          4966,   445,  2643, 14061]], device='cuda:0')

In [23]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


# 훈련된 LLM 토큰나이저 비교하기

| 항목                                             | 설명                             |
| ---------------------------------------------- | ------------------------------ |
| **colors_list**                                | 토큰을 구분하기 위해 배경색을 번갈아 사용할 색상 목록 |
| **tokenizer**                                  | 모델 전용 토크나이저 로드 (문장 → 토큰 ID)    |
| **token_ids**                                  | 문장을 구성하는 토큰들의 정수 ID 배열         |
| **decode(clean_up_tokenization_spaces=False)** | 공백/문자 정리를 하지 않고 토큰 그대로 복원      |
| **print 색상 코드**                                | 각 토큰을 색깔 배경으로 출력하여 시각적으로 확인    |



### 토크나이저 비교(Tokenizer Visualization)

- 실제 문장을 다양한 모델의 토크나이저로 분절해 보며 다음 개념 학습:

  - BERT(uncased/cased)

  - GPT-2

  - FLAN-T5

  - GPT-4 호환 토크나이저

  - StarCoder

  - Galactica

  - Phi-3

- 각 모델이 문장을 어떻게 subword 단위로 쪼개는지 색상 하이라이트로 시각화
- →
토크나이저에 따라 같은 문장도 토큰 구성이 크게 달라진다는 점을 이해

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# --------------------------------------------------------------
# 토큰들을 보기 좋게 색깔로 표시하기 위한 RGB 색상 목록입니다.
# 토큰 하나하나가 서로 다른 배경색을 갖도록 순서대로 사용됩니다.
# --------------------------------------------------------------
colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]


def show_tokens(sentence, tokenizer_name):
    # --------------------------------------------------------------
    # 1. 토크나이저 불러오기
    #    - tokenizer_name : 예) "bert-base-uncased", "gpt2", ...
    #    - 해당 모델에서 사용하는 토크나이저를 로드합니다.
    # --------------------------------------------------------------
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    # --------------------------------------------------------------
    # 2. 입력 문장(sentence)을 토큰 ID로 변환합니다.
    #    - tokenizer(sentence) → 토큰화 수행
    #    - input_ids : 문장을 구성하는 각 단어/조각에 해당하는 숫자 목록
    # --------------------------------------------------------------
    token_ids = tokenizer(sentence).input_ids

    # --------------------------------------------------------------
    # 3. 토큰 ID를 하나씩 출력합니다.
    #    enumerate(token_ids) : 토큰 번호(idx)와 실제 토큰 ID(t)를 가져옴
    # --------------------------------------------------------------
    for idx, t in enumerate(token_ids):

        # --------------------------------------------------------------
        # clean_up_tokenization_spaces=False 옵션 설명
        #
        # - tokenizer.decode()는 토큰을 다시 문자열로 바꾸는 함수입니다.
        # - 기본적으로 decode()는 공백을 자동으로 조정(clean-up)합니다.
        #   (예: '▁Hello' → 'Hello')
        #
        # - 하지만 “원본 토큰 그대로" 보고 싶을 때는
        #   clean_up_tokenization_spaces=False 로 설정해야 합니다.
        #
        # - transformers 4.45부터 기본값이 True(클린업 수행)로 변경될 예정.
        #
        # 자세한 이슈: https://github.com/huggingface/transformers/issues/31884
        # --------------------------------------------------------------

        # --------------------------------------------------------------
        # 4. 색깔을 적용한 상태로 토큰을 출력합니다.
        #    - \x1b[0;30;48;2;R;G;Bm  : 배경색 지정 ANSI 코드
        #    - idx % len(colors_list): 색상을 순환 적용
        #    - tokenizer.decode(t)    : 숫자 t를 다시 텍스트 토큰으로 변환
        #    - '\x1b[0m'              : 색상 초기화(원상 복귀)
        # --------------------------------------------------------------
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m'
            + tokenizer.decode(t, clean_up_tokenization_spaces=False)
            + '\x1b[0m',
            end=' '
        )

In [25]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"		" four spaces:"    "
12.0*50=600
"""

| 항목     | 설명                                              |
| ------ | ----------------------------------------------- |
| 함수명    | `show_tokens()`                                 |
| 입력값    | `text`(문장), `"bert-base-uncased"`(토크나이저 이름)     |
| 동작     | 텍스트를 토큰 ID로 변환 → 다시 텍스트로 복원                     |
| 출력 방식  | 각 토큰을 서로 다른 색깔 배경으로 출력                          |
| 목적     | BERT의 **토큰화 방식(Subword 분절 과정)** 을 시각적으로 이해하기 위함 |
| 시각적 효과 | 토큰을 색상으로 구분해 한눈에 인식 가능                          |


In [26]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

| 항목    | 설명                                           |
| ----- | -------------------------------------------- |
| 토크나이저 | `bert-base-cased` (대소문자 구분 유지)               |
| 입력    | `text` (분석할 문장)                              |
| 출력 형태 | 색깔 배경이 입혀진 토큰 문자열                            |
| 목적    | BERT의 **토큰 분절 방식(Subword Tokenization)** 시각화 |
| 특징    | “cased” 모델이므로 `Apple`과 `apple`을 다르게 처리       |
| 학습 효과 | 어떤 단어가 `##` 형태의 subword로 나뉘는지 직관적으로 확인 가능    |


- bert-base-cased 토크나이저를 불러옵니다.

  - “cased” 모델이므로 대소문자를 그대로 유지합니다.

  - 예: "Apple" 과 "apple" 을 다르게 처리함.

In [27]:
show_tokens(text, "bert-base-cased")

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

In [28]:
show_tokens(text, "gpt2")


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"        " 
 12 . 0 * 50 = 600 
 

In [29]:
show_tokens(text, "google/flan-t5-small")

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " four spaces : " " 12. 0 * 50 = 600  </s> 

In [30]:
# 공식 토크나이저는 `tiktoken`이지만 허깅 페이스 플랫폼에 동일한 토크나이저가 있습니다.
show_tokens(text, "Xenova/gpt-4")


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      "
 12 . 0 * 50 = 600 
 

In [31]:
show_tokens(text, "bigcode/starcoder2-15b")


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [32]:
show_tokens(text, "facebook/galactica-1.3b")


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : " 		 "  four  spaces : "      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [33]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :" 	 	 " four spaces :"    " 
 1 2 . 0 * 5 0 = 6 0 0 
 

# (BERT와 같은) 언어 모델로 문맥을 고려한 단어 임베딩 만들기

| 항목                  | 설명                                                |
| ------------------- | ------------------------------------------------- |
| 토크나이저               | 문장을 토큰(숫자)으로 변환 (`AutoTokenizer`)                 |
| 모델                  | DeBERTa v3 xsmall, 문장을 벡터로 인코딩하는 모델 (`AutoModel`) |
| tokens              | “Hello world”를 토큰 형태로 변환한 결과                      |
| return_tensors='pt' | PyTorch 텐서 형태로 반환                                 |
| model(**tokens)     | 토큰을 모델에 입력하여 특성 벡터(hidden states) 생성              |
| output              | 모델이 만든 은닉 상태(hidden representation), NLP 용 임베딩 벡터 |


In [34]:
from transformers import AutoModel, AutoTokenizer

# ---------------------------------------------------------
# 1. 토크나이저 불러오기
#    - "microsoft/deberta-v3-base" 모델에서 사용되는 토크나이저를 로드합니다.
#    - 토크나이저는 문장을 '토큰(token)'이라는 단위로 쪼개고,
#      이를 숫자로 변환해 모델이 이해할 수 있게 합니다.
# ---------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# ---------------------------------------------------------
# 2. 사전학습된 언어 모델 로드
#    - DeBERTa v3 계열 모델 중 'xsmall' 버전을 불러옵니다.
#    - AutoModel은 “언어 이해(encoding)”에 사용하는 모델을 로드합니다.
#      (문장을 벡터로 변환하는 용도)
# ---------------------------------------------------------
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# ---------------------------------------------------------
# 3. 문장을 토큰으로 변환
#    - 'Hello world'라는 문장을 tokenizer에 넣어
#      숫자 토큰으로 변환합니다.
#    - return_tensors='pt' → PyTorch 텐서로 반환
# ---------------------------------------------------------
tokens = tokenizer('Hello world', return_tensors='pt')

# ---------------------------------------------------------
# 4. 모델에 토큰을 입력해 처리
#    - **tokens 는 딕셔너리 형태로,
#      input_ids, attention_mask 등의 정보를 포함합니다.
#    - model(**tokens) : 딕셔너리를 한 번에 모델에 전달하는 방식
#    - [0] : 모델 출력 중 첫 번째 값(= hidden states)을 가져옵니다.
# ---------------------------------------------------------
output = model(**tokens)[0]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


| 차원                   | 의미                                        |
| -------------------- | ----------------------------------------- |
| 첫 번째 값 (1)           | 문장 개수(batch size)                         |
| 두 번째 값 (토큰 수)        | 입력 문장이 분해된 토큰의 개수                         |
| 세 번째 값 (hidden size) | 각 토큰을 표현하는 벡터 차원 수 (DeBERTa xsmall = 384) |
| 출력 전체 의미             | 문장의 모든 토큰을 “벡터 임베딩”으로 변환한 결과              |


In [35]:
output.shape

torch.Size([1, 4, 384])

In [36]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
world
[SEP]


In [37]:
output

tensor([[[-3.3060, -0.0507, -0.1098,  ..., -0.1704, -0.1618,  0.6932],
         [ 0.8918,  0.0740, -0.1583,  ...,  0.1869,  1.4760,  0.0751],
         [ 0.0871,  0.6364, -0.3050,  ...,  0.4729, -0.1829,  1.0157],
         [-3.1624, -0.1436, -0.0941,  ..., -0.0290, -0.1265,  0.7954]]],
       grad_fn=<NativeLayerNormBackward0>)

# 텍스트 임베딩 (문장과 전체 문서)

| 항목    | 설명                                        |
| ----- | ----------------------------------------- |
| 라이브러리 | `sentence-transformers` (문장 임베딩 특화 라이브러리) |
| 모델    | `all-mpnet-base-v2` (성능 매우 좋은 문장 임베딩 모델)  |
| 목적    | 문장의 의미를 벡터 형태로 변환                         |
| 입력    | `"Best movie ever!"`                      |
| 출력    | 768차원의 임베딩 벡터 (문장의 의미를 수치로 표현)            |
| 사용 용도 | 유사도 검색, 문장 비교, 클러스터링, 추천 시스템 등            |


In [38]:
from sentence_transformers import SentenceTransformer

# -------------------------------------------------------
# 1. Sentence-BERT(SBERT) 계열 모델 불러오기
#    - 'all-mpnet-base-v2'는 매우 성능 좋은 문장 임베딩 모델 중 하나입니다.
#    - 목적: 문장을 숫자 벡터(임베딩)로 변환해 의미 비교가 가능하게 만드는 것
# -------------------------------------------------------
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# -------------------------------------------------------
# 2. 텍스트 → 임베딩 벡터로 변환하기
#    - model.encode()에 문장을 넣으면
#      문장의 의미를 표현하는 고차원 벡터로 변환됩니다.
#    - 예: "Best movie ever!" → [0.123, -0.045, 0.998, ...] (768차원)
# -------------------------------------------------------
vector = model.encode("Best movie ever!")

In [39]:
vector.shape

(768,)

In [ ]:
vector

# LLM 밖의 단어 임베딩


In [41]:
# gensim을 설치합니다. 설치 후 코랩 런타임 세션을 다시 시작해 주세요.
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 49.4 MB/s eta 0:00:00


| 항목     | 설명                                   |
| ------ | ------------------------------------ |
| 라이브러리  | Gensim (전통적 단어 임베딩을 다루는 라이브러리)       |
| 모델 이름  | `"glove-wiki-gigaword-50"`           |
| 학습 데이터 | Wikipedia + Gigaword                 |
| 임베딩 차원 | 50차원                                 |
| 특징     | 단어 기반 임베딩(GloVe), fast & lightweight |
| 활용     | 단어 유사도 계산, 단어 벡터 탐색, 간단 NLP 실습       |


In [42]:
import gensim.downloader as api

# ---------------------------------------------------------------
# 1. 사전학습된 단어 임베딩(Word Embedding) 다운로드
#
#   - gensim은 대표적인 단어 임베딩 라이브러리입니다.
#   - api.load()를 사용하면 유명한 임베딩 모델을 바로 다운로드할 수 있습니다.
#
#   여기서 사용하는 모델:
#   "glove-wiki-gigaword-50"
#     → GloVe 임베딩(GloVe: Global Vectors)
#     → 위키백과(Wikipedia) & Gigaword 데이터로 학습됨
#     → 각 단어를 50차원의 벡터로 표현
#
#   ※ 다른 선택지 예:
#     - "word2vec-google-news-300" (300차원, 구글 뉴스로 학습)
#     - "glove-wiki-gigaword-100" (100차원 버전)
#
#   더 많은 임베딩 목록:
#   https://github.com/RaRe-Technologies/gensim-data
# ---------------------------------------------------------------
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [43]:
# king과 가장 의미적으로 비슷한 단어 11개를 찾아줘
model.most_similar([model['king']], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]

# 임베딩으로 노래 추천하기

| 항목              | 설명                            |
| --------------- | ----------------------------- |
| `train.txt`     | 여러 곡의 ID로 이루어진 플레이리스트 데이터     |
| 앞 2줄 제거         | 메타데이터라서 분석에서 제외               |
| `playlists`     | 노래가 2곡 이상 포함된 재생목록만 필터링       |
| `song_hash.txt` | 노래의 ID, 제목, 아티스트 정보를 담고 있는 파일 |
| `songs_df`      | 노래 메타데이터를 담고 있는 pandas 데이터프레임 |
| 목적              | 재생목록 분석 / 추천 시스템 실습 준비 데이터    |


### Word2Vec 기반 음악 추천 시스템 실습

- 재생목록 데이터를 사용한 추천 시스템 구축 과정

  1. Playlist 데이터 불러오기

  2. Song metadata 로드 (title, artist)

  3. Word2Vec 학습

  4. 특정 노래와 가장 유사한 노래 추천

In [44]:
import pandas as pd
from urllib import request

# ---------------------------------------------------------
# 1. 재생목록(playlist) 데이터셋 다운로드
#    - Google Cloud에 저장된 train.txt 파일을 불러옵니다.
#    - 이 파일에는 여러 개의 재생목록 정보가 들어 있습니다.
# ---------------------------------------------------------
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# ---------------------------------------------------------
# 2. 파일 읽기 및 파싱
#    - 데이터를 UTF-8 문자열로 변환한 뒤, 줄 단위로 나눕니다.
#    - 앞의 첫 두 줄은 메타데이터(설명)만 들어 있으므로 건너뜁니다.
# ---------------------------------------------------------
lines = data.read().decode("utf-8").split('\n')[2:]

# ---------------------------------------------------------
# 3. 재생목록 데이터 정제
#    - 각 줄은 "노래 ID들"로 이루어진 하나의 재생목록입니다.
#    - split()으로 공백 기준 나누고,
#    - 노래가 하나만 있는 재생목록은 의미가 없으므로 제외합니다.
# ---------------------------------------------------------
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# ---------------------------------------------------------
# 4. 노래 정보(메타데이터) 다운로드
#    - song_hash.txt 파일에는
#        '노래 ID', '제목', '가수명'
#      정보가 탭( \t )으로 구분되어 있습니다.
# ---------------------------------------------------------
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')

# ---------------------------------------------------------
# 5. 노래 메타데이터를 리스트로 변환
# ---------------------------------------------------------
songs = [s.rstrip().split('\t') for s in songs_file]

# ---------------------------------------------------------
# 6. 판다스 DataFrame으로 변환
#    - ID, 제목(title), 가수명(artist) 열을 가진 표(table)로 만들기
# ---------------------------------------------------------
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])

# ---------------------------------------------------------
# 7. 노래 ID를 인덱스로 설정
#    - songs_df.loc["노래ID"] 형태로 쉽게 접근 가능
# ---------------------------------------------------------
songs_df = songs_df.set_index('id')

In [45]:
print('재생목록 #1:\n ', playlists[0], '\n')
print('재생목록 #2:\n ', playlists[1])

재생목록 #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

재생목록 #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117', '118', 

| 항목          | 설명                                     |
| ----------- | -------------------------------------- |
| 학습 데이터      | `playlists` = 사용자 재생목록(노래 ID 목록)       |
| 모델 종류       | Word2Vec (Skip-gram/CBOW 기반 전통적 임베딩)   |
| vector_size | 노래 벡터 차원 = 32                          |
| window      | 한 목록에서 양쪽 20곡까지 문맥으로 사용                |
| negative    | 네거티브 샘플링 50개 사용                        |
| min_count   | 1번 이상 등장한 노래 모두 포함                     |
| workers     | CPU 스레드 4개로 병렬 처리                      |
| 목적          | **재생목록 기반 “노래 임베딩” 학습 → 음악 추천 시스템 기초** |


In [46]:
from gensim.models import Word2Vec

# ---------------------------------------------------------
# Word2Vec 모델 학습
# ---------------------------------------------------------
# playlists : 우리가 위에서 만든 재생목록 데이터
#             예: [["song1", "song25", "song99"], ["song3", "song66", ...], ...]
#             즉, 각 리스트는 한 명의 사용자가 함께 들은 노래들의 집합
#
# Word2Vec 아이디어:
#   - 함께 나오는 단어(여기서는 '노래 ID')는 의미적으로 비슷하다.
#   - → 재생목록에 자주 함께 등장하는 노래들은 임베딩 공간에서 가까워짐
#
# 모델 주요 설정:
#   vector_size=32 : 각 노래를 32차원 벡터로 표현
#   window=20      : 한 재생목록 안에서 주변 20곡까지 '문맥'으로 간주
#   negative=50    : 네거티브 샘플링 개수 (학습 속도 향상 기법)
#   min_count=1    : 1번 이상 등장한 노래는 모두 학습에 포함
#   workers=4      : CPU 코어 4개 사용해 병렬 학습
# ---------------------------------------------------------
model = Word2Vec(
    playlists,
    vector_size=32,
    window=20,
    negative=50,
    min_count=1,
    workers=4
)

In [47]:
song_id = 2172

# 노래 ID 2172와 비슷한 노래를 찾으라고 모델에게 요청합니다.
model.wv.most_similar(positive=str(song_id))

[('3167', 0.9979429244995117),
 ('3094', 0.9975169897079468),
 ('2849', 0.9961586594581604),
 ('2881', 0.9958118796348572),
 ('2976', 0.9955345392227173),
 ('6624', 0.9952554702758789),
 ('2904', 0.9952357411384583),
 ('3151', 0.9946845173835754),
 ('2640', 0.9943594336509705),
 ('2715', 0.9934158325195312)]

In [48]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


| 항목           | 설명                                           |
| ------------ | -------------------------------------------- |
| 함수 목적        | Word2Vec 임베딩 기반으로 특정 노래와 비슷한 노래 추천           |
| 입력값          | `song_id` (추천을 시작할 기준 노래 ID)                 |
| most_similar | Word2Vec이 학습한 벡터 공간에서 “가장 가까운 5개 노래” 찾기      |
| 문자열 변환       | Word2Vec 토큰은 문자열이므로 `str(song_id)` 사용        |
| 출력           | 추천된 노래 5개의 **제목 + 아티스트 정보가 담긴 표(DataFrame)** |
| 기본 아이디어      | “함께 재생목록에 자주 등장하는 노래는 서로 비슷하다”는 Word2Vec 원리  |


In [51]:
import numpy as np

def print_recommendations(song_id):
    # ---------------------------------------------------------
    # model.wv.most_similar()
    #   - 학습된 Word2Vec 임베딩에서
    #   - 특정 노래(song_id)와 가장 비슷한 노래 5개를 찾습니다.
    #
    # positive=str(song_id)
    #   - Word2Vec은 문자열로 된 토큰을 사용하므로
    #     song_id(숫자)를 문자열로 변환해 전달합니다.
    #
    # 결과 형태:
    #   [('1234', 0.82), ('5521', 0.80), ... ]
    #   → 이 중 첫 번째 요소(노래 ID)들만 추출합니다.
    # ---------------------------------------------------------
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id), topn=5)
    )[:, 0]   # 유사한 노래의 ID만 추출 (5개)

    # ---------------------------------------------------------
    # songs_df는 노래 메타데이터(title, artist 등)를 담고 있는 데이터프레임
    # iloc[ID 목록]을 사용하면 해당 노래들의 제목·가수 정보가 표로 반환됩니다.
    # ---------------------------------------------------------
    return songs_df.iloc[similar_songs]

In [56]:
# ---------------------------------------------------------
# 예시: 노래 ID 2172와 비슷한 노래 추천하기
# ---------------------------------------------------------
print(songs_df.iloc[2172])
print_recommendations(2172)

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


,title,artist
id,,
3167,Unchained,Van Halen
3094,Breaking The Law,Judas Priest
2849,Run To The Hills,Iron Maiden
2881,Drop Dead Legs,Van Halen
2976,I Don't Know,Ozzy Osbourne


In [57]:
print(songs_df.iloc[842])
print_recommendations(842)

title     California Love (w\/ Dr. Dre & Roger Troutman)
artist                                              2Pac
Name: 842 , dtype: object


,title,artist
id,,
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
374,Jook Gal,Elephant Man
1418,Tick Tock,Kesha
5788,Drop It Like It's Hot (w\/ Pharrell),Snoop Dogg
18844,Murder She Wrote,Chaka Demus & Pliers
